# 08 - Spark: Debugging & Optimization

This notebook covers Spark-specific utilities for debugging and optimizing pipelines.

| Part | Topic |
|------|-------|
| **1** | CpuInfo - Know Your Workers |
| **2** | LogDataSkew - Monitor Partition Distribution |
| **3** | Detecting Skew in Split Pipelines |
| **4** | Worker Package Diagnostics |

⚠️ **This notebook requires a Spark session.** Examples show expected output but won't run without PySpark.

In [1]:
# Spark session setup (adjust for your environment)
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Nebula Spark Demo") \
    .master("local[4]") \
    .getOrCreate()

from nebula import TransformerPipeline
from nebula.transformers.spark_transformers import CpuInfo, LogDataSkew, Repartition
from nebula.transformers import AddLiterals, Filter, SelectColumns

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 16:00:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create sample data with 10K rows
from pyspark.sql import functions as F

df = spark.range(10000).withColumn(
    "category", 
    F.when(F.col("id") < 100, "tiny")      # 100 rows (1%)
     .when(F.col("id") < 500, "small")     # 400 rows (4%)
     .when(F.col("id") < 2000, "medium")   # 1500 rows (15%)
     .otherwise("large")                    # 8000 rows (80%)
).withColumn(
    "value", (F.rand() * 1000).cast("int")
)

df = df.repartition(8)  # Start with 8 balanced partitions
df.groupBy("category").count().show()

[Stage 0:>                                                          (0 + 4) / 4]

+--------+-----+
|category|count|
+--------+-----+
|    tiny|  100|
|  medium| 1500|
|   small|  400|
|   large| 8000|
+--------+-----+



---
## Part 1: CpuInfo - Know Your Workers

It requires `cpuinfo`, `pip install py-cpuinfo`.

`CpuInfo` runs a UDF across workers to report their CPU model. This is useful when:

- You're on a **managed cluster** (Databricks, EMR, Dataproc) and want to verify hardware
- You suspect **heterogeneous workers** are causing performance issues
- You want to **blame infrastructure** when things are slow 😄

### 1.1 Basic Usage

In [3]:
# Run once at pipeline start to see what you're working with
cpu_checker = CpuInfo()
df_with_cpu = cpu_checker.transform(df)

# The transformer adds no columns - it just logs CPU info
# Check your Spark logs for output like:
# 
# *** CpuInfo ***
# Intel(R) Xeon(R) CPU @ 2.20GHz: 4 workers
# AMD EPYC 7B12: 2 workers

/Users/vamarett2101/Desktop/oss/nebula/venv/lib/python3.10/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
[Stage 8:===================================================>    (92 + 4) / 100]

+----------------------------------------+---------------+
|cpu                                     |host           |
+----------------------------------------+---------------+
|Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz|CHLC02FN46PMD6T|
+----------------------------------------+---------------+



In [4]:
# Typically placed at the start of a pipeline
pipe = TransformerPipeline(
    [
        CpuInfo(),  # First thing: what hardware do we have?
        SelectColumns(columns=["id", "category", "value"]),
        # ... rest of pipeline
    ],
    name="Pipeline with CPU Check",
)

pipe.show()

*** Pipeline with CPU Check *** (2 transformations)
 - CpuInfo
 - SelectColumns


---
## Part 2: LogDataSkew - Monitor Partition Distribution

`LogDataSkew` reports how rows are distributed across partitions. Skewed partitions are a common cause of slow Spark jobs - one task takes forever while others finish quickly.

### 2.1 Basic Usage

In [5]:
# Check partition distribution
skew_checker = LogDataSkew()
df_checked = skew_checker.transform(df)

# Output in logs:
# 
# *** LogDataSkew ***
# Partitions: 8
# Total rows: 10000
# Min/Max rows per partition: 1200 / 1300
# Skew ratio: 1.08x (healthy < 2x)

2025-12-27 16:01:28,218 | [INFO]: Number of partitions: 8 
2025-12-27 16:01:28,219 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:01:28,219 | [INFO]: mean=1,250.0 | std=0.8 | min=1,249.0 | 25%=1,249.8 | 50%=1,250.0 | 75%=1,250.2 | max=1,251.0 


### 2.2 As Interleaved Transformer

The real power comes from using `LogDataSkew` as an **interleaved transformer** - it runs after every step, showing how skew evolves through your pipeline.

In [6]:
# Monitor skew at every step
pipe = TransformerPipeline(
    [
        SelectColumns(columns=["id", "category", "value"]),
        Filter(input_col="value", perform="keep", operator="gt", value=100),
        AddLiterals(data=[{"alias": "processed", "value": True}]),
    ],
    name="Pipeline with Skew Monitoring",
    interleaved=[LogDataSkew()],  # Runs after each transformer
)

pipe.show()

# When you run this, logs show skew after EACH step:
# Running 'SelectColumns' ...
# *** LogDataSkew *** Partitions: 8, Skew: 1.08x
# Running 'Filter' ...
# *** LogDataSkew *** Partitions: 8, Skew: 1.12x
# ...
pipe.run(df)

2025-12-27 16:01:28,226 | [INFO]: Starting pipeline 'Pipeline with Skew Monitoring' 
2025-12-27 16:01:28,227 | [INFO]: Running 'SelectColumns' ... 
2025-12-27 16:01:28,249 | [INFO]: Completed 'SelectColumns' in 0.0s 
2025-12-27 16:01:28,250 | [INFO]: Running 'LogDataSkew' ... 


*** Pipeline with Skew Monitoring *** (5 transformations)
 - SelectColumns
 - LogDataSkew
 - Filter
 - LogDataSkew
 - AddLiterals


2025-12-27 16:01:28,610 | [INFO]: Number of partitions: 8 
2025-12-27 16:01:28,611 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:01:28,612 | [INFO]: mean=1,250.0 | std=0.8 | min=1,249.0 | 25%=1,249.8 | 50%=1,250.0 | 75%=1,250.2 | max=1,251.0 
2025-12-27 16:01:28,612 | [INFO]: Completed 'LogDataSkew' in 0.4s 
2025-12-27 16:01:28,613 | [INFO]: Running 'Filter' ... 
2025-12-27 16:01:28,625 | [INFO]: Completed 'Filter' in 0.0s 
2025-12-27 16:01:28,625 | [INFO]: Running 'LogDataSkew' ... 
2025-12-27 16:01:29,041 | [INFO]: Number of partitions: 8 
2025-12-27 16:01:29,042 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:01:29,043 | [INFO]: mean=1,119.8 | std=0.5 | min=1,119.0 | 25%=1,119.8 | 50%=1,120.0 | 75%=1,120.0 | max=1,120.0 
2025-12-27 16:01:29,043 | [INFO]: Completed 'LogDataSkew' in 0.4s 
2025-12-27 16:01:29,044 | [INFO]: Running 'AddLiterals' ... 
2025-12-27 16:01:29,054 | [INFO]: Completed 'AddLiterals' in 0.0s 
2025-12-27 16:01:29,054 | [INFO]: Pipeline 'Pipel

DataFrame[id: bigint, category: string, value: int, processed: boolean]

### 2.3 Runtime Injection with `force_interleaved_transformer`

Don't want to modify your pipeline definition? Inject `LogDataSkew` at runtime:

In [7]:
# Original pipeline - no skew monitoring
production_pipe = TransformerPipeline(
    [
        SelectColumns(columns=["id", "category", "value"]),
        Filter(input_col="value", perform="keep", operator="gt", value=100),
    ],
    name="Production Pipeline",
)

# Inject skew monitoring at runtime (for debugging)
result = production_pipe.run(
    df,
    force_interleaved_transformer=LogDataSkew(),
)

# Production code stays clean, debugging when needed

2025-12-27 16:01:29,067 | [INFO]: Starting pipeline 'Production Pipeline' 
2025-12-27 16:01:29,069 | [INFO]: Running 'SelectColumns' ... 
2025-12-27 16:01:29,390 | [INFO]: Number of partitions: 8 
2025-12-27 16:01:29,391 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:01:29,391 | [INFO]: mean=1,250.0 | std=0.8 | min=1,249.0 | 25%=1,249.8 | 50%=1,250.0 | 75%=1,250.2 | max=1,251.0 
2025-12-27 16:01:29,392 | [INFO]: Completed 'SelectColumns' in 0.3s 
2025-12-27 16:01:29,393 | [INFO]: Running 'Filter' ... 
2025-12-27 16:01:29,720 | [INFO]: Number of partitions: 8 
2025-12-27 16:01:29,721 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:01:29,722 | [INFO]: mean=1,119.8 | std=0.5 | min=1,119.0 | 25%=1,119.8 | 50%=1,120.0 | 75%=1,120.0 | max=1,120.0 
2025-12-27 16:01:29,722 | [INFO]: Completed 'Filter' in 0.3s 
2025-12-27 16:01:29,723 | [INFO]: Pipeline 'Production Pipeline' completed in 0.7s 


---
## Part 3: Detecting Skew in Split Pipelines

Split pipelines are a common source of skew - when you split data by category and then append, partition distribution often becomes unbalanced.

### 3.1 Creating a Skewed Pipeline

In [8]:
from pyspark.sql import functions as F

def split_by_category(df):
    """Split into 4 very unequal subsets."""
    return {
        "tiny": df.filter(F.col("category") == "tiny"),      # 100 rows (1%)
        "small": df.filter(F.col("category") == "small"),    # 400 rows (4%)
        "medium": df.filter(F.col("category") == "medium"),  # 1500 rows (15%)
        "large": df.filter(F.col("category") == "large"),    # 8000 rows (80%)
    }

In [9]:
skewed_pipeline = TransformerPipeline(
    {
        "tiny": [
            AddLiterals(data=[{"alias": "priority", "value": "critical"}]),
        ],
        "small": [
            AddLiterals(data=[{"alias": "priority", "value": "high"}]),
        ],
        "medium": [
            AddLiterals(data=[{"alias": "priority", "value": "normal"}]),
        ],
        "large": [
            AddLiterals(data=[{"alias": "priority", "value": "batch"}]),
        ],
    },
    split_function=split_by_category,
    name="Category Processing",
)

skewed_pipeline.show()

*** Category Processing *** (4 transformations)
------ SPLIT ------ (function: split_by_category)
**SPLIT <<< large >>> (1 transformation):
     - AddLiterals
**SPLIT <<< medium >>> (1 transformation):
     - AddLiterals
**SPLIT <<< small >>> (1 transformation):
     - AddLiterals
**SPLIT <<< tiny >>> (1 transformation):
     - AddLiterals
<<< Append DFs >>>


### 3.2 Monitoring the Skew

In [10]:
full_pipeline = TransformerPipeline(
    [
        CpuInfo(),  # What hardware?
        
        # This split will create skew
        skewed_pipeline,
        
        # After append, partitions are unbalanced
        SelectColumns(columns=["id", "category", "value", "priority"]),
    ],
    name="Full Pipeline with Skew",
)

# Run with skew monitoring
result = full_pipeline.run(
    df,
    force_interleaved_transformer=LogDataSkew(),
)

# Expected log output:
#
# Running 'CpuInfo' ...
# *** CpuInfo *** Intel Xeon: 4 workers
# *** LogDataSkew *** Partitions: 8, Skew: 1.08x ✓
#
# Entering split ...
# ... (split processing) ...
# <<< Append DFs >>>
# *** LogDataSkew *** Partitions: 32, Skew: 80x ⚠️  <- PROBLEM!
#
# Running 'SelectColumns' ...
# *** LogDataSkew *** Partitions: 32, Skew: 80x ⚠️

2025-12-27 16:01:29,744 | [INFO]: Starting pipeline 'Full Pipeline with Skew' 
2025-12-27 16:01:29,745 | [INFO]: Running 'CpuInfo' ... 
/Users/vamarett2101/Desktop/oss/nebula/venv/lib/python3.10/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
                                                                                

+----------------------------------------+---------------+
|cpu                                     |host           |
+----------------------------------------+---------------+
|Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz|CHLC02FN46PMD6T|
+----------------------------------------+---------------+



2025-12-27 16:02:11,999 | [INFO]: Number of partitions: 8 
2025-12-27 16:02:12,000 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:02:12,001 | [INFO]: mean=1,250.0 | std=0.8 | min=1,249.0 | 25%=1,249.8 | 50%=1,250.0 | 75%=1,250.2 | max=1,251.0 
2025-12-27 16:02:12,001 | [INFO]: Completed 'CpuInfo' in 42.3s 
2025-12-27 16:02:12,002 | [INFO]: Entering split 
2025-12-27 16:02:12,020 | [INFO]: Running 'AddLiterals' ... 
2025-12-27 16:02:12,374 | [INFO]: Number of partitions: 8 
2025-12-27 16:02:12,374 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:02:12,375 | [INFO]: mean=1,000.0 | std=0.8 | min=999.0 | 25%=999.8 | 50%=1,000.0 | 75%=1,000.2 | max=1,001.0 
2025-12-27 16:02:12,377 | [INFO]: Completed 'AddLiterals' in 0.4s 
2025-12-27 16:02:12,379 | [INFO]: Running 'AddLiterals' ... 
2025-12-27 16:02:12,730 | [INFO]: Number of partitions: 8 
2025-12-27 16:02:12,730 | [INFO]: Rows distribution in partitions: 
2025-12-27 16:02:12,731 | [INFO]: mean=187.5 | std=0.5 | min=187

### 3.3 Fixing the Skew

Use `repartition_output_to_original` or `coalesce_output_to_original`:

In [11]:
fixed_pipeline = TransformerPipeline(
    {
        "tiny": [AddLiterals(data=[{"alias": "priority", "value": "critical"}])],
        "small": [AddLiterals(data=[{"alias": "priority", "value": "high"}])],
        "medium": [AddLiterals(data=[{"alias": "priority", "value": "normal"}])],
        "large": [AddLiterals(data=[{"alias": "priority", "value": "batch"}])],
    },
    split_function=split_by_category,
    name="Category Processing (Fixed)",
    repartition_output_to_original=True,  # Rebalance after append
)

# Now skew is fixed after the split
# *** LogDataSkew *** Partitions: 8, Skew: 1.25x ✓

---
## Part 4: Worker Package Diagnostics

When you don't manage the Spark cluster directly (Databricks, EMR, company clusters), package version mismatches between driver and workers cause cryptic errors.

### 4.1 Check if a Package is Installed

In [12]:
from nebula.spark_udfs import lib_in_spark_workers, lib_version_in_spark_workers

# Check if pandas is available on workers
has_pandas = lib_in_spark_workers(spark, "pandas")
print(f"pandas installed on workers: {has_pandas}")

# Check a package that might not be there
has_xgboost = lib_in_spark_workers(spark, "xgboost")
print(f"xgboost installed on workers: {has_xgboost}")

[Stage 67:=============================>                            (2 + 2) / 4]

pandas installed on workers: True
xgboost installed on workers: False


### 4.2 Check Package Version

In [13]:
# Get the version installed on workers
pandas_version = lib_version_in_spark_workers(spark, "pandas")
print(f"Worker pandas version: {pandas_version}")

# Compare with driver
import pandas as pd
print(f"Driver pandas version: {pd.__version__}")

# Mismatch? That might explain your UDF errors!

[Stage 69:===========================================>              (3 + 1) / 4]

Worker pandas version: 2.3.3
Driver pandas version: 2.3.3


### 4.3 Diagnostic Pattern

When debugging cluster issues:

In [14]:
def diagnose_cluster(spark, packages: list[str]):
    """Quick cluster diagnostic."""
    print("=" * 50)
    print("CLUSTER DIAGNOSTICS")
    print("=" * 50)
    
    for pkg in packages:
        installed = lib_in_spark_workers(spark, pkg)
        if installed:
            version = lib_version_in_spark_workers(spark, pkg)
            print(f"✓ {pkg}: {version}")
        else:
            print(f"✗ {pkg}: NOT INSTALLED")
    
    print("=" * 50)


# Usage
diagnose_cluster(spark, ["pandas", "numpy", "pyarrow", "nebula"])

CLUSTER DIAGNOSTICS


✓ pandas: 2.3.3
✓ numpy: 2.2.6
✓ pyarrow: 22.0.0
✓ nebula: Unable to determine the package version


---
## Summary

| Tool | Purpose | When to Use |
|------|---------|-------------|
| `CpuInfo` | Report worker CPU types | Start of pipeline, debugging performance |
| `LogDataSkew` | Monitor partition distribution | Interleaved or after joins/splits |
| `lib_in_spark_workers` | Check package availability | Debugging UDF failures |
| `lib_version_in_spark_workers` | Check package versions | Driver/worker version mismatch |

**Skew Prevention:**
- Use `repartition_output_to_original=True` on split pipelines
- Monitor with `LogDataSkew` as interleaved transformer
- Inject `force_interleaved_transformer=LogDataSkew()` at runtime for debugging

**Cluster Debugging:**
- When UDFs fail mysteriously, check package versions
- Driver version ≠ worker version is a common issue on managed clusters

In [15]:
spark.stop()